# Transfer Learning

In [1]:
"""
1. VGG16
2. ResNet50
3. Inseption
4. Xception
"""

'\n1. VGG16\n2. ResNet50\n3. Inseption\n4. Xception\n'

In [3]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import numpy as np

In [4]:
!unzip "/content/drive/MyDrive/Animal Data/Animal_Dataset.zip"

Archive:  /content/drive/MyDrive/Animal Data/Animal_Dataset.zip
   creating: dataset/
   creating: dataset/Testing/
   creating: dataset/Testing/bears/
  inflating: dataset/Testing/bears/k4 (100).jpeg  
  inflating: dataset/Testing/bears/k4 (100).jpg  
  inflating: dataset/Testing/bears/k4 (101).jpeg  
  inflating: dataset/Testing/bears/k4 (101).jpg  
  inflating: dataset/Testing/bears/k4 (102).jpeg  
  inflating: dataset/Testing/bears/k4 (102).jpg  
  inflating: dataset/Testing/bears/k4 (103).jpeg  
  inflating: dataset/Testing/bears/k4 (104).jpeg  
  inflating: dataset/Testing/bears/k4 (105).jpeg  
  inflating: dataset/Testing/bears/k4 (106).jpeg  
  inflating: dataset/Testing/bears/k4 (107).jpeg  
  inflating: dataset/Testing/bears/k4 (108).jpeg  
  inflating: dataset/Testing/bears/k4 (109).jpeg  
  inflating: dataset/Testing/bears/k4 (110).jpeg  
  inflating: dataset/Testing/bears/k4 (71).jpg  
  inflating: dataset/Testing/bears/k4 (72).jpeg  
  inflating: dataset/Testing/bears/k4 

In [5]:
train_path = "/content/dataset/Training"
test_path = "/content/dataset/Testing"

In [6]:
train_gen = ImageDataGenerator(rescale=1./255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True)

test_gen = ImageDataGenerator(rescale=1./255) 

In [7]:
train = train_gen.flow_from_directory(train_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

test = test_gen.flow_from_directory(test_path,
                                      target_size=(224,224),
                                      batch_size=22,
                                      class_mode='categorical')

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


# VGG16

In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

In [9]:
# Adding the preprocessing layer to the front of vgg

vgg = VGG16(include_top=False,weights='imagenet',input_shape=(224,224,3))

58889256/58889256 [==============================] - 2s 0us/step


In [10]:
# Train model with existing weights

for layer in vgg.layers:
  print(layer)

In [11]:
# Train model with existing weights

for layer in vgg.layers:
  layer.trainable=False

In [12]:
x = Flatten()(vgg.output)

In [13]:
# output layer

prediction = Dense(4,activation='softmax')(x)

In [14]:
# Create Vgg16 model

model = Model(inputs=vgg.input,outputs=prediction)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
model.fit_generator(train,validation_data=test,epochs=4,steps_per_epoch=len(train),
                    validation_steps=len(test))

<ipython-input-17-0aa3deeaa555>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train,validation_data=test,epochs=4,steps_per_epoch=len(train),


Epoch 1/4
57/57 [==============================] - 38s 447ms/step - loss: 0.7305 - accuracy: 0.7399 - val_loss: 0.1275 - val_accuracy: 0.9816
Epoch 2/4
57/57 [==============================] - 23s 398ms/step - loss: 0.1706 - accuracy: 0.9491 - val_loss: 0.0704 - val_accuracy: 0.9877
Epoch 3/4
57/57 [==============================] - 22s 382ms/step - loss: 0.1278 - accuracy: 0.9653 - val_loss: 0.0370 - val_accuracy: 0.9969
Epoch 4/4
57/57 [==============================] - 22s 383ms/step - loss: 0.0751 - accuracy: 0.9798 - val_loss: 0.0271 - val_accuracy: 0.9939


# ResNet50

In [19]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [20]:
resnet = ResNet50(include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 4s 0us/step


In [21]:
for layer in resnet.layers:
  layer.trainable=False

In [22]:
for layer in resnet.layers:
  print(layer)

In [24]:
x = Flatten()(resnet.output)

In [25]:
out = Dense(4, activation='softmax')(x)

In [26]:
res_model = Model(inputs=resnet.input,outputs=out)

In [27]:
res_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [28]:
res_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
res_model.fit(train,epochs=5,validation_data=test,steps_per_epoch=len(train),
              validation_steps=len(test))

Epoch 1/5
57/57 [==============================] - 32s 473ms/step - loss: 2.8363 - accuracy: 0.4103 - val_loss: 1.2220 - val_accuracy: 0.5184
Epoch 2/5
57/57 [==============================] - 22s 378ms/step - loss: 1.2623 - accuracy: 0.4879 - val_loss: 0.8631 - val_accuracy: 0.6503
Epoch 3/5
57/57 [==============================] - 24s 419ms/step - loss: 1.1548 - accuracy: 0.5614 - val_loss: 0.7346 - val_accuracy: 0.6840
Epoch 4/5
57/57 [==============================] - 26s 455ms/step - loss: 1.9128 - accuracy: 0.4968 - val_loss: 1.1401 - val_accuracy: 0.5644
Epoch 5/5
57/57 [==============================] - 22s 394ms/step - loss: 1.5523 - accuracy: 0.5412 - val_loss: 1.1742 - val_accuracy: 0.5337
